In [3]:
import json
import time

class Context:
    def __init__(self, entities, relationships, current_activities, current_events):
        self.entities = entities
        self.relationships = relationships
        self.activities = current_activities
        self.events = current_events
        self.history_events = []
        self.history_activities = []
    
    
#kiedy event dodawany jest do historii? wtedy, kiedy sie odbywa
#gdy zachodzi nowy event, co sie dzieje? do czasu jego zakonczenia jest current eventem a potem przechodzi do historii?

    def update_events(new_events):  
        self.history_events.append(new_events)
        self.current_events = new_events
        
    def __str__(self):
        to_print = 'Entities :' + str(self.entities) + ', relationships: ' + str(self.relationships) + \
        ',  activities: ' + str(self.activities) + ', events: ' + str(self.events)
        return to_print
    
# ----------------------------------------------------------------------------------------------------------
        
class Entity:
    def __init__(self, entity_id, entity_type, internal_id, entity_name):
        self.entity_id = entity_id
        self.entity_type = entity_type
        self.internal_id = internal_id
        self.entity_name = entity_name
        
    def __str__(self):
        to_print = 'Entity id:' + str(self.entity_id) + ', entity name: ' + str(self.entity_name)
        return to_print
    
def entity_decoder(obj):
    if '__type__' in obj and obj['__type__'] == 'Entity':
        return Entity(obj['entity_id'], obj['entity_type'], obj['internal_id'], obj['entity_name'])
    return obj

def load_entities(json_list):
    entities = []
    for element in json_list:
        ent = entity_decoder(element)
        entities.append(ent)
    return entities

# -------------------------------------------------------------------------------------------------------

class Relationship:
    def __init__(self, relationship_id, relationship_type, relationship_name, ENT1, ENT2, relationship_state):
        self.relationship_id = relationship_id
        self.relationship_type = relationship_type
        self.relationship_name = relationship_name
        self.ENT1 = ENT1
        self.ENT2 = ENT2
        self.relationship_state = relationship_state
        
        
def relationship_decoder(obj):
    if '__type__' in obj and obj['__type__'] == 'Relationship':
        return Relationship(obj['relationship_id'], obj['relationship_type'], obj['relationship_name'], obj['ENT1'], obj['ENT2'], obj['relationship_state'])
    return obj


def load_relationships(json_list):
    relationships = []
    for element in json_list:
        rel = relationship_decoder(element)
        relationships.append(rel)
    return relationships


# -------------------------------------------------------------------------------------------------------
class Activity:
    def __init__(self, activity_id, activity_type, activity_result,  activity_entity_id):
        self.activity_id = activity_id
        self.activity_type = activity_type
        self.activity_result = activity_result
        self.activity_time_start = activity_time_start
        self.activity_time_end = activity_time_end
        self.activity_entity_id = activity_entity_id
        
        
def activity_decoder(obj):
    if '__type__' in obj and obj['__type__'] == 'Activity':
        return Activity(obj['activity_id'], obj['activity_type'], obj['activity_result'], obj['activity_entity_id'])
    return obj


def load_activities(json_list):
    activities = []
    for element in json_list:
        act = activity_decoder(element)
        activities.append(act)
    return activities



#----------------------------------------------------- EVENT ---------------------------------------------------
 
class Event:
    def __init__(self, event_id, event_name, event_goal, event_location, event_entities, event_activities, event_state):
        self.event_id = event_id
        self.event_name = event_name
        self.event_goal = event_goal
        self.event_location = event_location
        self.event_entities = event_entities
        self.event_activities = event_activities
        self.event_state = event_state
        
        
def event_decoder(obj):
    if '__type__' in obj and obj['__type__'] == 'Event':
        return Event(obj['event_id'], obj['event_name'], obj['event_goal'], obj['event_location'],\
                     obj['event_entities'], obj['event_activities'], \
                     obj['event_state'])
    return obj


def load_events(json_list):
    events = []
    for element in json_list:
        ev = event_decoder(element)
        events.append(ev)
    return events

In [4]:


class RuleGenerator:
    def __init__(self, event_name, event_goal, event_entities, event_activities):
        self.event_name = event_name
        self.event_goal = event_goal
        self.event_entities = event_entities
        self.event_activities = event_activities
        self.activities_result = ""
        self.relationships = []
        
    def __str__(self):
        to_print = '\nRule:  event name: ' + str(self.event_name) + ', event goal: ' + str(self.event_goal)+ \
        ', event entities: ' + str(self.event_entities) + ', event activities: ' + str(self.event_activities) +\
        ', activities result: ' + str(self.activities_result)  + ', relationships: ' 
        
        rel_list = ''
        for element in self.relationships:
            sub_string = '\nRel: ' +  str(element)
            rel_list += sub_string
        
        return to_print + rel_list + ' \n'
    
        
def define_rules(context_list):
    for element in context_list:
        events_list = element.events
        entities_list = element.entities
        relationships_list = element.relationships
        activities_list = element.activities
        
        print("Generating rules...\n\n")
        time.sleep(5)
        
        rules = []
        
        for event in events_list:
            ev_name = event["event_name"]
            ev_goal = event["event_goal"]
            ev_entities = event["event_entities"]
            ev_activities = event["event_activities"]
            new_rule = RuleGenerator(ev_name, ev_goal, ev_entities, ev_activities)
            rules.append(new_rule)
            
            
        # do tego trzeba pobrac result z aktywnoscci - done
      
        found_results_of_activities = []
        for activity in activities_list:
            act_id = activity["activity_id"]
            act_type = activity["activity_type"]
            act_result = activity["activity_result"]
            
            for rule in rules:
                act_set = set(rule.event_activities)
                
                if act_id in act_set:
                    found_results_of_activities.append(act_result)
        
        
        for i in range (len(found_results_of_activities)):
            rules[i].activities_result = found_results_of_activities[i] 

        
        found_relationships_of_entities = []
        
        #tworzenie slownika do badania zagniezdzonych podmiotow
        
        entities_to_investigate_relations = {}
        for i in range(len(rules)):
            entities_to_investigate_relations[i] = []
      
        
        for relationship in relationships_list:
            found_relationships_of_entity = []
            rel_id = relationship["relationship_id"]
            rel_ent1 = relationship["ENT1"]
            rel_ent2 = relationship["ENT2"]
            
            for i in range(len(rules)):
                rule_ent = rules[i].event_entities[0]
                
                if rule_ent == rel_ent1 or rule_ent == rel_ent2:
                    found_relationships_of_entity.append(rel_id) # dla uproszczenia
                    #dodaj kolejne znalezione podmioty, zeby tez je sprawdzic
                    """
                    if rule_ent == rel_ent1 :
                        entities_to_investigate_relations[i].append(rel_ent2)
                    else:
                        entities_to_investigate_relations[i].append(rel_ent1)
                    """ 
            if found_relationships_of_entity:
                found_relationships_of_entities.append(found_relationships_of_entity)
                
        #print('found_relationships_of_entities: ', found_relationships_of_entities)
        
        
        
        for i in range (len(found_relationships_of_entities)):
            rules[i].relationships = found_relationships_of_entities[i] 
            
            
            
            
        """      
            print("slownik ze znalezionymi kolejnymi podmiotami: ", entities_to_investigate_relations)    
                
                    
            if found_relationships_of_entity:
                found_relationships_of_entities.append(found_relationships_of_entity)
        
        print("found_relationships_of_entities:  ", found_relationships_of_entities)
        #jeszcze raz szukamy relacji
        for relationship in relationships_list:
            found_relationships_of_entity = []
            rel_id = relationship["relationship_id"]
            rel_ent1 = relationship["ENT1"]
            rel_ent2 = relationship["ENT2"]
            
            for i in range(len(rules)):
                rule_ent = entities_to_investigate_relations[i][0]
                
                if rule_ent == rel_ent1 or rule_ent == rel_ent2:
                    found_relationships_of_entity.append(rel_id) # dla uproszczenia   
                
                    
            if found_relationships_of_entity:
                found_relationships_of_entities.append(found_relationships_of_entity)
        """      
            
        
        for rule in rules:
            
            print("\n\nIf: \n")
            
            to_print = 'Entities: ' + str(rule.event_entities) + \
            '\nAND: Activities: ' + str(rule.event_activities) +\
            '\nAND: Result: >' + str(rule.activities_result)  + ' Celsius degrees ' + \
            '\nAND: Relationships: ' + str(rule.relationships)  + \
            '\n\nThen: ' + str(rule.event_name) + '\n\nDo: ' + str(rule.event_goal)
            
            print(to_print)
            
            
        

        """
        
        
        print("\n\nIf: \n")
        
        for entity in entities_list:
            ent_id = entity["entity_id"]
            print("Present entity: ", ent_id)
            
            ent_type = entity["entity_type"]
            print('Of type: ', ent_type)
        
        print("And: \n")
        
        for relationship in relationships_list:
            
            rel_id = relationship["relationship_id"]
            rel_type = relationship["relationship_type"]
            rel_name = relationship["relationship_name"]
            print("With relationship: ", rel_id, ", of type: ", rel_type, ", named: ", rel_name)
            
        print("And activities: \n")
        
        
        
        for event in events_list:
            alert = event["event_name"]
            action = event["event_goal"]
            print('Then: ', alert)
            print('Do: ', action) """


In [5]:
entities_jsons = [
{
 "entity_id" : "ENT-KONTENER-001", 
 "entity_type" : "Kontener", 
 "internal_id" : "1",
 "entity_name": "Kontener nr 1"
},
{
 "entity_id" : "ENT-KONTENER-002", 
 "entity_type" : "Kontener",
 "internal_id" : "2",
 "entity_name": "Kontener nr 2"
},
{
 "entity_id" : "ENT-CZUJNIK-001", 
 "entity_type" : "Czujnik", 
 "internal_id" : "1",
 "entity_name": "Czujnik nr 1"
},
{
 "entity_id" : "ENT-CZUJNIK-002", 
 "entity_type" : "Czujnik",
 "internal_id" : "2",
 "entity_name": "Czujnik nr 2"
},
{
 "entity_id" : "ENT-CZUJNIK-003", 
 "entity_type" : "Czujnik",
 "internal_id" : "3",
 "entity_name": "Czujnik nr 3"
},
{
 "entity_id" : "ENT-PALIWO-001", 
 "entity_type" : "Paliwo-klasa1",
 "internal_id" : "1",
 "entity_name": "Paliwo nr 1 o klasie zaplonu nr 1"
},
{
 "entity_id" : "ENT-PALIWO-002", 
 "entity_type" : "Paliwo-klasa2",
 "internal_id" : "2",
 "entity_name": "Paliwo nr 2 o klasie zaplonu nr 2"
}
]

relationship_jsons = [
{
 "relationship_id" : "REL_ENT-CZUJNIK-001_INSIDE_ENT-KONTENER-001_0001",
 "relationship_type" : "ENT1 inside ENT2",
 "relationship_name" : "Czujnik nr 1 znajduje się w Kontenerze nr 1",
 "ENT1": "ENT-CZUJNIK-001",
 "ENT2" : "ENT-KONTENER-001",
 "relationship_state" : "active"
},
{
 "relationship_id" : "REL_ENT-CZUJNIK-002_INSIDE_ENT-KONTENER-001_0002",
 "relationship_type" : "ENT1 inside ENT2",
 "relationship_name" : "Czujnik nr 2 znajduje się w Kontenerze nr 1",
 "ENT1": "ENT-CZUJNIK-002",
 "ENT2" : "ENT-KONTENER-001",
 "relationship_state" : "active"
},
{
 "relationship_id" : "REL_ENT-CZUJNIK-003_INSIDE_ENT-KONTENER-002_0003",
 "relationship_type" : "ENT1 inside ENT2",
 "relationship_name" : "Czujnik nr 3 znajduje się w Kontenerze nr 2",
 "ENT1": "ENT-CZUJNIK-003",
 "ENT2" : "ENT-KONTENER-002",
 "relationship_state" : "active"
},
{
 "relationship_id" : "REL_ENT-PALIWO-001_INSIDE_ENT-KONTENER-001_0004",
 "relationship_type" : "ENT1 inside ENT2",
 "relationship_name" : "Paliwo nr 1 znajduje się w Kontenerze nr 1",
 "ENT1": "ENT-PALIWO-001",
 "ENT2" : "ENT-KONTENER-001",
 "relationship_state" : "active"
},
{
 "relationship_id" : "REL_ENT-PALIWO-002_INSIDE_ENT-KONTENER-002_0005",
 "relationship_type" : "ENT1 inside ENT2",
 "relationship_name" : "Paliwo nr 2 znajduje się w Kontenerze nr 2",
 "ENT1": "ENT-PALIWO-002",
 "ENT2" : "ENT-KONTENER-002",
 "relationship_state" : "active"
}
]

activities_jsons = [
{
 "activity_id" : "ACT-POMIAR_TEMPERATURY-001",
 "activity_type" : "pomiar_temperatury",
 "activity_result": "40",
 "activity_entity_id": "ENT-CZUJNIK-001"
},
{
 "activity_id" : "ACT-POMIAR_TEMPERATURY-002",
 "activity_type" : "pomiar_temperatury",
 "activity_result": "25",
 "activity_entity_id": "ENT-CZUJNIK-003"
}
]
    
events_jsons = [
{
 "event_id": "EVT-ALERT-001",
 "event_name": "Alert nr 1 - najwyższy stopień zagrożenia",
 "event_goal": "Powiadom ochrone, straz pożarną i wojsko",
 "event_location" : "ul. Warszawska 15A, 42-200 Częstochowa, I piętro, pierwszy kontener od okna", 
 "event_entities" : ["ENT-CZUJNIK-001", "ENT-KONTENER-001", "ENT-PALIWO-001"],
 "event_activities" :  ["ACT-POMIAR_TEMPERATURY-001"],
 "event_state" : "active"
},
{
 "event_id": "EVT-ALERT-001",
 "event_name": "Alert nr 3 - średni stopień zagrozenia",
 "event_goal": "Powiadom ochrone",
 "event_location" : "ul. Warszawska 15A, 42-200 Częstochowa, I piętro, pierwszy kontener od okna", 
 "event_entities" : ["ENT-CZUJNIK-003", "ENT-KONTENER-002", "ENT-PALIWO-002"],
 "event_activities" :  ["ACT-POMIAR_TEMPERATURY-002"],
 "event_state" : "active"
}
]
# "event_entities" : ["ENT-CZUJNIK-001", "ENT-KONTENER-001"],
#"event_entities" : ["ENT-CZUJNIK-003", "ENT-KONTENER-002", "ENT-PALIWO-002"],


# aktywnosci jako input, eventy jako output i na tej podstawie reguly

entities = load_entities(entities_jsons)    
relationships = load_relationships(relationship_jsons)
activities = load_activities(activities_jsons)
events = load_events(events_jsons)
    
new_context = Context(entities, relationships, activities, events)
#print(new_context)

rules_based_on_context = define_rules([new_context])


Generating rules...




If: 

Entities: ['ENT-CZUJNIK-001', 'ENT-KONTENER-001', 'ENT-PALIWO-001']
AND: Activities: ['ACT-POMIAR_TEMPERATURY-001']
AND: Result: >40 Celsius degrees 
AND: Relationships: ['REL_ENT-CZUJNIK-001_INSIDE_ENT-KONTENER-001_0001']

Then: Alert nr 1 - najwyższy stopień zagrożenia

Do: Powiadom ochrone, straz pożarną i wojsko


If: 

Entities: ['ENT-CZUJNIK-003', 'ENT-KONTENER-002', 'ENT-PALIWO-002']
AND: Activities: ['ACT-POMIAR_TEMPERATURY-002']
AND: Result: >25 Celsius degrees 
AND: Relationships: ['REL_ENT-CZUJNIK-003_INSIDE_ENT-KONTENER-002_0003']

Then: Alert nr 3 - średni stopień zagrozenia

Do: Powiadom ochrone
